In [3]:
import numpy 
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split


C:\Users\Kanha\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df=pd.read_csv('movie_review.csv', delimiter=',')

In [5]:
df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos


In [6]:
df.describe

<bound method NDFrame.describe of        fold_id cv_tag  html_id  sent_id  \
0            0  cv000    29590        0   
1            0  cv000    29590        1   
2            0  cv000    29590        2   
3            0  cv000    29590        3   
4            0  cv000    29590        4   
...        ...    ...      ...      ...   
64715        9  cv999    14636       20   
64716        9  cv999    14636       21   
64717        9  cv999    14636       22   
64718        9  cv999    14636       23   
64719        9  cv999    14636       24   

                                                    text  tag  
0      films adapted from comic books have had plenty...  pos  
1      for starters , it was created by alan moore ( ...  pos  
2      to say moore and campbell thoroughly researche...  pos  
3      the book ( or " graphic novel , " if you will ...  pos  
4      in other words , don't dismiss this film becau...  pos  
...                                                  ...  ...  
6

In [7]:
df.shape

(64720, 6)

In [8]:
df.replace('pos', 1, inplace=True)
df.replace('neg', 0, inplace=True)

In [9]:
df.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,1
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",1
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,1
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",1
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",1


In [10]:
df1=df[df['tag']==1].head(1000)
df2=df[df['tag']==0].head(1000)

In [11]:
df1.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,1
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",1
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,1
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",1
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",1


In [12]:
df2.head()

,fold_id,cv_tag,html_id,sent_id,text,tag
32937,0,cv000,29416,0,"plot : two teen couples go to a church party ,...",0
32938,0,cv000,29416,1,they get into an accident .,0
32939,0,cv000,29416,2,"one of the guys dies , but his girlfriend cont...",0
32940,0,cv000,29416,3,what's the deal ?,0
32941,0,cv000,29416,4,"watch the movie and "" sorta "" find out . . .",0


In [13]:
df=pd.concat([df1,df2])

In [14]:
df.head(1010)

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,1
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",1
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,1
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",1
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",1
...,...,...,...,...,...,...
32942,0,cv000,29416,5,critique : a mind-fuck movie for the teen gene...,0
32943,0,cv000,29416,6,which is what makes this review an even harder...,0
32944,0,cv000,29416,7,they seem to have taken this pretty neat conce...,0
32945,0,cv000,29416,8,so what are the problems with the movie ?,0


In [15]:
df.shape

(2000, 6)

In [16]:
df.reset_index(inplace=True)

In [17]:
df.head(1003)

,index,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,0,cv000,29590,0,films adapted from comic books have had plenty...,1
1,1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",1
2,2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,1
3,3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",1
4,4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",1
...,...,...,...,...,...,...,...
998,998,0,cv032,22550,7,""" in the heat of the night "" was the first and...",1
999,999,0,cv032,22550,8,jewison has a very keen style of displaying va...,1
1000,32937,0,cv000,29416,0,"plot : two teen couples go to a church party ,...",0
1001,32938,0,cv000,29416,1,they get into an accident .,0


In [18]:
df=df.iloc[:,5:7]
df

,text,tag
0,films adapted from comic books have had plenty...,1
1,"for starters , it was created by alan moore ( ...",1
2,to say moore and campbell thoroughly researche...,1
3,"the book ( or "" graphic novel , "" if you will ...",1
4,"in other words , don't dismiss this film becau...",1
...,...,...
1995,hong kong cinema has been going through a bad ...,0
1996,the last few productions have been effect lade...,0
1997,"in a nutshell , the current crop of films from...",0
1998,with the one time british colony reverting bac...,0


In [19]:
model_class, tokenizer_class, pretrained_weights= (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

In [20]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [21]:
tokenized = df['text'].apply((lambda x : tokenizer.encode(x, add_special_tokens=True, padding=True, truncation=True)))
                                                           
                                                          

max_len = max(len(seq) for seq in tokenized)

for i in tokenized:
    i.extend([0]*(max_len - len(i)))
    
input_ids = torch.tensor(tokenized)
with torch.no_grad():
    last_hidden_states = model(input_ids)
    
features = last_hidden_states[0][:,0,:].numpy()

labels = df['tag']
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

logistic_regression_classifier = LogisticRegression()
logistic_regression_classifier.fit(train_features, train_labels)

logistic_regression_classifier.score(test_features, test_labels)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
C:\Users\Kanha\AppData\Roaming\Python\Python311\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.688

In [22]:
!mkdir '/kaggle/working/saved_models'
save_directory = "/kaggle/working/saved_models" 

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

The syntax of the command is incorrect.


('/kaggle/working/saved_models\\tokenizer_config.json',
 '/kaggle/working/saved_models\\special_tokens_map.json',
 '/kaggle/working/saved_models\\vocab.txt',
 '/kaggle/working/saved_models\\added_tokens.json')

In [80]:
tokenizer_fine_tuned = ppb.DistilBertTokenizer.from_pretrained(save_directory)
model_fine_tuned = ppb.DistilBertForSequenceClassification.from_pretrained(save_directory)

test_text = "i'm huge jim carrey fan"
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'pt'    
)

output = model_fine_tuned(predict_input)[0]

prediction_value = torch.argmax(output, axis = 1).numpy()[0]

prediction_value

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at /kaggle/working/saved_models and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1